In [ ]:
import serial
import matplotlib.pyplot as plt
import time
import pandas as pd
from lib import *
import numpy as np
from numpy import sqrt, log, cos, tan, degrees
from scipy.signal import step, lti, TransferFunction
from cmath import polar

In [ ]:
df = pd.read_csv('data/data.csv')
T = 0.0001 
samples = int(1/T/100)
df['segundos'] = df.index*T
tempo = df['segundos'].values
valores = df['Channel 1'].values
fig, ax = plt.subplots()
ax.plot(tempo, valores)

In [ ]:
max_value = max(df['Channel 1'])
df['diff'] = abs(df['Channel 1'] - max_value*0.63)
i = df.loc[df['Channel 1'] == 0.0].index[-1]
time1 = float(df.loc[df['diff'] == min(df['diff']), 'segundos'])
time0 = df.loc[i,'segundos']
time_last = time1 + T*samples
tau = float(time1 - time0)
K = int(df['Channel 1'].max())/127
time0,time1,time_last, tau

In [ ]:
num = [max_value]
den = [tau, 1]
t_total = time_last - time0

In [ ]:
t = np.linspace(0,t_total,samples)
y = transfer_function(num, den, t)
plt.plot(t, y)
plt.xlabel('Tempo (s)')
plt.ylabel('Amplitude')
plt.title('Resposta a Degrau da Função Transferência')
plt.show()

In [ ]:
x1 = df.loc[df['segundos']>= time0, 'segundos'] - time0
y1 = df.loc[df['segundos']>= time0, 'Channel 1']

x2 = t
y2 = y
# Cria uma figura\n",
fig, ax = plt.subplots()
# Adiciona o primeiro gráfico à figura\n",
ax.plot(x1, y1, label='Real')
# Adiciona o segundo gráfico à figura\n",
ax.plot(x2, y2, label='Simulado')

# Adiciona legenda\n",
ax.legend()
# Mostra a figura\n",
plt.show()

In [ ]:
Tp = 2*tau
mp = 0.1
Kamort = sqrt((log(mp)**2)/(np.pi**2+(log(mp)**2)))
omega_d = np.pi/Tp
omega_n = omega_d/sqrt(1-Kamort**2)
p1 = complex (Kamort*omega_n, omega_d)
p2 = complex (Kamort*omega_n, -omega_d)
Hs0 = K/(tau*p1+1)

In [ ]:
rHs0, tetaHs0 = polar(Hs0)
rp , tetap = polar(p1)
tetacs0 = np.pi - tetaHs0
rcs0 = 1/rHs0

In [ ]:
A = (1/p1).real
B = (1/p1).imag
Ti = 1/(B/tan(tetacs0)-A)
k = 1/(rHs0*abs(1+1/(Ti*p1)))
ki = k/Ti

In [ ]:
kp = (2*omega_n*Kamort*tau-1)/K
ki = omega_n**2*tau/K
kp,ki